In [1]:
import sys

my_list = [1, 2, 3]
print(sys.getrefcount(my_list))

another_ref = my_list
print(sys.getrefcount(my_list))

another_ref = None
print(sys.getrefcount(my_list))



2
3
2


In [2]:
import gc
import weakref

class Node:
    def __init__(self, value):
        self.value = value
        self.children = []
        self.parent = None

    def add_child(self, child):
        child.parent = self
        self.children.append(child)

parent = Node("parent")
child = Node("child")
parent.add_child(child)

print(f"Objects before: {len(gc.get_objects())}")

collected = gc.collect()
print(f"Objects collected: {collected}")

def callback(ref):
    print("Object was garbage collected!")


weak_ref = weakref.ref(parent, callback)
parent = None
child = None
gc.collect()

Objects before: 77526
Objects collected: 371
Object was garbage collected!


4

In [3]:
import gc
import sys
from collections import defaultdict

def memory_usage():
    stats = defaultdict(int)
    for obj in gc.get_objects():
        stats[type(obj).__name__] += 1
    return dict(stats)

def find_memory_leaks():
    gc.set_debug(gc.DEBUG_LEAK)

    data = []
    for i in range(1000):
        data.append([i] * 100)

    collected = gc.collect()
    if gc.garbage:
        print(f"Memory leaks detected: {len(gc.garbage)} objects")
        for obj in gc.garbage:
            print(f"Leaked object: {type(obj)}")

    return collected

print("Memory before:", memory_usage())
find_memory_leaks()
print("Memory after:", memory_usage())

Memory before: {'dict': 9378, 'IPythonDisplayFormatter': 1, 'list': 5363, 'MimeBundleFormatter': 1, 'PlainTextFormatter': 1, 'HTMLFormatter': 2, 'MarkdownFormatter': 1, 'SVGFormatter': 1, 'PNGFormatter': 1, 'PDFFormatter': 1, 'JPEGFormatter': 1, 'LatexFormatter': 1, 'JSONFormatter': 1, 'JavascriptFormatter': 1, 'cell': 2482, 'tuple': 10865, 'function': 18495, 'HistoryOutput': 3, 'defaultdict': 18, 'coroutine': 14, 'Future': 8, 'builtin_function_or_method': 1418, 'TimerHandle': 1, 'Context': 29, 'FutureIter': 3, 'Frame': 7, 'Handle': 11, 'method': 2401, 'partial': 14, 'Condition': 92, 'RLock': 16, 'deque': 52, 'hamt_bitmap_node': 3, 'hamt': 3, '_GeneratorContextManager': 5, 'generator': 5, 'ExecutionInfo': 2, 'ExecutionResult': 2, 'ReferenceType': 3795, 'Module': 2, 'Import': 2, 'alias': 3, 'ImportFrom': 1, 'FunctionDef': 2, 'arguments': 2, 'Assign': 3, 'Name': 34, 'Call': 17, 'For': 3, 'Attribute': 9, 'AugAssign': 1, 'Subscript': 1, 'Constant': 8, 'Return': 2, 'Expr': 7, 'List': 43, 'B

In [ ]:
import weakref

class ExpensiveResource:
    def __init__(self, name):
        self.name = name
        print(f"Creating expensive resource: {name}")

    def __del__(self):
        print(f"Destroying resource: {self.name}")

resource = ExpensiveResource("database")
regular_ref = resource


weak_ref = weakref.ref(resource)

print(f"Weak reference alive: {weak_ref() is not None}")

resource = None
regular_ref = None

import gc
gc.collect()
print(f"Weak reference alive: {weak_ref() is not None}")